## Tim's Models

Consists of: CatBoost, Random Forests, Linear Regressor, K-Nearest-Neighbour.

In [4]:
from catboost import CatBoostRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
import pandas as pd
import numpy as np
import seaborn as sns

In [5]:
# Load data
train = pd.read_csv('final_train.csv')
test = pd.read_csv('final_test.csv')

# Check data
print("Training set ", train.shape)
print("Test set ", test.shape)

Training set  (3000, 3738)
Test set  (4398, 5048)


In [6]:
# Fill remaining NA's with 0 and negatives with 0
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)
train[train < 0] = 0
test[test < 0] = 0

In [7]:
# Drop ID Column
train.drop(['id'], axis=1, inplace=True)
test.drop(['id'], axis=1, inplace=True)

In [8]:
train.describe()

,budget,popularity,runtime,revenue,name_collection,budget_log,genre_Drama,genre_Comedy,genre_Thriller,genre_Action,...,prodc_Quick Six Entertainment,production_countries_count,release_month,release_day,release_year,spoken_languages_count,is_released,keywords_count,cast_score,crew_score
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,...,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,11.959606,1.793891,4.648978,15.970738,1.026360,2.037452,0.510333,0.342667,0.263000,0.247000,...,0.000333,0.203871,1.722570,2.440693,7.600729,0.261150,0.998667,1.621991,20.144832,17.074726
std,7.405678,0.863576,0.347045,3.045649,2.088409,1.241217,0.499977,0.474680,0.440336,0.431339,...,0.018257,0.385399,0.701168,0.875534,0.007766,0.437214,0.036497,0.923586,2.661406,2.503348
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,7.560601,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.390797,4.543295,14.691625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.386294,2.079442,7.597396,0.000000,1.000000,1.098612,19.608674,16.530195
50%,15.894952,1.998076,4.644391,16.637310,0.000000,2.766002,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.945910,2.708050,7.602900,0.000000,1.000000,1.791759,20.567880,17.739403
75%,17.216708,2.387935,4.770685,18.046365,0.000000,2.845880,1.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,2.302585,3.091042,7.606387,0.693147,1.000000,2.302585,21.361155,18.439050
max,19.755682,5.684725,5.823046,21.141685,6.045005,2.983441,1.000000,1.000000,1.000000,1.000000,...,1.000000,2.079442,2.484907,3.433987,7.609367,2.197225,1.000000,5.003946,25.925794,20.645642


In [9]:
y = train.revenue
X = train.drop('revenue', axis=1)

In [7]:
z = train.sort_values('revenue', ascending=False)
z['revenue']

1126    21.141685
1761    21.132889
2770    21.063590
684     20.956666
2322    20.839934
          ...    
695      0.693147
1917     0.000000
1874     0.000000
1754     0.000000
347      0.000000
Name: revenue, Length: 3000, dtype: float64

In [8]:
print(train['budget'].corr(train['revenue']))

0.5345104574056031


In [10]:
# Select Top 50 Best Features
number_of_features = 50
best_features = SelectKBest(score_func=chi2, k=number_of_features)
y = y.astype('int')
fit = best_features.fit(X, y)
df_scores = pd.DataFrame(fit.scores_)
df_columns = pd.DataFrame(X.columns)
feature_scores = pd.concat([df_columns, df_scores], axis=1)
feature_scores.columns = ['Specs', 'Score']
print(feature_scores.nlargest(number_of_features, 'Score'))

                                                  Specs        Score
0                                                budget  4390.389619
3                                       name_collection  1631.334064
2644        prodc_Québec Production Services Tax Credit   999.000000
2645                    prodc_Abu Dhabi Film Commission   999.000000
2646  prodc_Colorado Office of Film, Television & Media   999.000000
1160                         prodc_Chongoing Film Group   749.000000
1161                       prodc_Bon Voyage Film Studio   749.000000
1162                         prodc_Shanghai Media Group   749.000000
1163      prodc_Zhejiang Films & TV(Group) Company Ltd.   749.000000
1164                    prodc_Hunan Broadcasting System   749.000000
1165                     prodc_Anhui Broadcasting Corp.   749.000000
1166                           prodc_Beijing TV Station   749.000000
1858                      prodc_Centerstage Productions   749.000000
1859  prodc_Film Development Counc

In [11]:
selected_features = feature_scores.nlargest(number_of_features, 'Score')['Specs'].tolist()

In [12]:
X = X[selected_features]
X.describe()

,budget,name_collection,prodc_Québec Production Services Tax Credit,prodc_Abu Dhabi Film Commission,"prodc_Colorado Office of Film, Television & Media",prodc_Chongoing Film Group,prodc_Bon Voyage Film Studio,prodc_Shanghai Media Group,prodc_Zhejiang Films & TV(Group) Company Ltd.,prodc_Hunan Broadcasting System,...,prodc_Pandemonium,prodc_Lightstream Entertainment,prodc_iFeatures,prodc_Sixty Six Pictures,prodc_Oldgarth Media,prodc_Кинокомпания «Lunapark»,prodc_Инвада фильм,prodc_ABS-CBN Film Productions,prodc_Unitel Classics,prodc_Adel Productions
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,...,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,11.959606,1.026360,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,...,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333
std,7.405678,2.088409,0.018257,0.018257,0.018257,0.018257,0.018257,0.018257,0.018257,0.018257,...,0.018257,0.018257,0.018257,0.018257,0.018257,0.018257,0.018257,0.018257,0.018257,0.018257
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15.894952,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,17.216708,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,19.755682,6.045005,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=.2, random_state=13)

In [13]:
print('Training: ', X_tr.shape)
print('Validation: ', X_val.shape)
print('Test: ', test.shape)

Training:  (2400, 50)
Validation:  (600, 50)
Test:  (4398, 5047)


In [14]:
def kfold_validate(X, y, te, model, col):
    scores = np.zeros(k)
    for i, (tr_idx, val_idx) in enumerate(cv.split(X)):
        X_tr, y_tr = X.reindex(tr_idx), y.reindex(tr_idx)
        X_val, y_val = X.reindex(val_idx), y.reindex(val_idx)
        
        X_tr.fillna(-1, inplace=True)
        y_tr.fillna(-1, inplace=True)
        X_val.fillna(-1, inplace=True)
        y_val.fillna(-1, inplace=True)
        

        model.fit(X_tr, y_tr)
        pred_val = model.predict(X_val)
        scores[i] = get_loss(pred_val, y_val)
        print("========={}-th Fold Score: {}".format(i, scores[i]))
        
        meta_tr[col][val_idx] = pred_val
        meta_tr['val'][val_idx] = y_val

    print("=========Total Score: ", np.mean(scores))
    model.fit(X, y)
    meta_te[col] = model.predict(te)

In [15]:
def get_loss(pred, actual):
  loss = mean_squared_error(pred, actual)
  result = np.sqrt(loss)
  return result

In [16]:
k = 5
seed = 10
cv = KFold(n_splits = k, shuffle = True, random_state = seed)

In [17]:
meta_tr = pd.DataFrame(np.zeros((X.shape[0], 8)),
                            columns=['catboost', 'random_forest', 'KNN', 'linear_regression', 'ridge_regression', 'support_vector_regression', 'elastic_net', 'xgboost'])
meta_tr['val'] = 0
meta_te = pd.DataFrame(np.zeros((test.shape[0], 8)),
                            columns=['catboost', 'random_forest', 'KNN', 'linear_regression', 'ridge_regression', 'support_vector_regression', 'elastic_net', 'xgboost'])

### CatBoost

In [18]:
cat_boost = CatBoostRegressor(bagging_temperature = 0.3, loss_function='RMSE', logging_level='Silent',
                             colsample_bylevel = 0.7, depth = 9, early_stopping_rounds = 200, iterations = 1000, eval_metric='RMSE', learning_rate = 0.01)
kfold_validate(X_tr, y_tr, test, cat_boost, 'catboost')

=========0-th Fold Score: 1.7367833182456593


/Users/shuyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/shuyang/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/shuyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


=========1-th Fold Score: 1.5145875945002159
=========2-th Fold Score: 1.4091197444475019
=========3-th Fold Score: 1.7717750440922657
=========4-th Fold Score: 1.7931683930622975
=========Total Score:  1.6450868188695877


CatBoostError: catboost/libs/data/model_dataset_compatibility.cpp:79: At position 1 should be feature with name name_collection (found popularity).

In [28]:
cat_boost_pred = cat_boost.predict(X_val)
np.sqrt(mean_squared_error(y_val, cat_boost_pred))

1.949315376306365

### Random Forest

In [29]:
random_forest = RandomForestRegressor(n_estimators = 3000, max_depth = 9, criterion='mse')
kfold_validate(X_tr, y_tr, test, random_forest, 'random_forest')

=========0-th Fold Score: 1.8751736663268717
=========1-th Fold Score: 1.533313098066968
=========2-th Fold Score: 1.390662952323086
=========3-th Fold Score: 1.8007238914925892
=========4-th Fold Score: 1.8596381679831828
=========Total Score:  1.6919023552385397


ValueError: Number of features of the model must match the input. Model n_features is 50 and input n_features is 5047 

In [30]:
random_forest_pred = random_forest.predict(X_val)
np.sqrt(mean_squared_error(y_val, random_forest_pred))

1.9604190762207157

### Linear Regression -- WORST PERFORMER

In [31]:
linear_reg = LinearRegression()
kfold_validate(X_tr, y_tr, test, linear_reg, 'linear_regression')

=========0-th Fold Score: 2.2267011842489546
=========1-th Fold Score: 2.083412716183726
=========2-th Fold Score: 2.1588715533725944
=========3-th Fold Score: 2.6524886460148225
=========4-th Fold Score: 2.2501395239807485
=========Total Score:  2.2743227247601694


ValueError: shapes (4398,5047) and (50,) not aligned: 5047 (dim 1) != 50 (dim 0)

In [32]:
lin_res = linear_reg.predict(X_val)
np.sqrt(mean_squared_error(y_val, lin_res))

49286010823.361694

### K-NN

In [33]:
knn = KNeighborsRegressor(n_neighbors=10, weights='distance', p=5)
kfold_validate(X_tr, y_tr, test, knn, 'KNN')

=========0-th Fold Score: 1.8353990168759093
=========1-th Fold Score: 1.7632745818128033
=========2-th Fold Score: 1.4218632848515094
=========3-th Fold Score: 1.8368592587314498
=========4-th Fold Score: 1.7574376447052056
=========Total Score:  1.7229667573953755


ValueError: query data dimension must match training data dimension

In [34]:
knn_pred = knn.predict(X_val)
np.sqrt(mean_squared_error(y_val, knn_pred))

2.134488765827412

###### Shu is working on Ridge Regression, Support Vector Regression, Elastic Net, Lightgbm, XGBoost

## Ridge Regression

In [19]:
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt

#find the alpha with best value(here we choose 0.001)
alphas = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1,0.5, 1]
for a in alphas:
    model = Ridge(alpha=a, normalize=True).fit(X,y) 
    score = model.score(X, y)
    pred_y = model.predict(X)
    mse = mean_squared_error(y, pred_y) 
    print("Alpha:{0:.6f}, R2:{1:.3f}, MSE:{2:.2f}, RMSE:{3:.2f}".format(a, score, mse, np.sqrt(mse)))

Alpha:0.000001, R2:0.617, MSE:3.56, RMSE:1.89
Alpha:0.000010, R2:0.617, MSE:3.56, RMSE:1.89
Alpha:0.000100, R2:0.617, MSE:3.56, RMSE:1.89
Alpha:0.001000, R2:0.616, MSE:3.57, RMSE:1.89
Alpha:0.010000, R2:0.602, MSE:3.70, RMSE:1.92
Alpha:0.100000, R2:0.584, MSE:3.87, RMSE:1.97
Alpha:0.500000, R2:0.559, MSE:4.10, RMSE:2.02
Alpha:1.000000, R2:0.522, MSE:4.44, RMSE:2.11


In [20]:
ridge_reg=Ridge(alpha=0.001, normalize=True).fit(X_tr,y_tr)
ypred = ridge_reg.predict(X_val)
score = model.score(X_val,y_val)
mse = mean_squared_error(y_val,ypred)
print("R2:{0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}".format(score, mse,np.sqrt(mse))) 

R2:0.528, MSE:4.57, RMSE:2.14


In [21]:
kfold_validate(X_tr, y_tr, test, ridge_reg, 'ridge_regression')

=========0-th Fold Score: 2.114747660594332
=========1-th Fold Score: 1.9565309535378355
=========2-th Fold Score: 1.9175857969669352
=========3-th Fold Score: 2.3716624651516973
=========4-th Fold Score: 2.1550676882497575
=========Total Score:  2.1031189129001113


ValueError: shapes (4398,5047) and (50,) not aligned: 5047 (dim 1) != 50 (dim 0)

In [22]:
rid_res = ridge_reg.predict(X_val)
np.sqrt(mean_squared_error(y_val, rid_res))

2.1381223460156145

## Support Vector Regression

In [23]:
from sklearn.svm import SVR

support_vector_reg = SVR(kernel='rbf',C=2.0, epsilon=0.2)
kfold_validate(X_tr, y_tr, test, support_vector_reg, 'support_vector_regression')

/Users/shuyang/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


=========0-th Fold Score: 1.8472225343050794


/Users/shuyang/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


=========1-th Fold Score: 1.5294713504019017


/Users/shuyang/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


...

=========3-th Fold Score: 1.825507225972868


/Users/shuyang/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


=========4-th Fold Score: 1.8628407189661658
=========Total Score:  1.7025430919775801


/Users/shuyang/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


ValueError: X.shape[1] = 5047 should be equal to 50, the number of features at training time

In [24]:
support_vector_res = support_vector_reg.predict(X_val)
np.sqrt(mean_squared_error(y_val, support_vector_res))

2.0111878034062993

## Elastic Net

In [25]:
from sklearn.linear_model import ElasticNet

#find the alpha with best value(here we choose 0.001)
alphas = [0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 1]
for a in alphas:
    model = ElasticNet(alpha=a, normalize=True).fit(X,y) 
    score = model.score(X, y)
    pred_y = model.predict(X)
    mse = mean_squared_error(y, pred_y) 
    print("Alpha:{0:.6f}, R2:{1:.3f}, MSE:{2:.2f}, RMSE:{3:.2f}".format(a, score, mse, np.sqrt(mse)))

Alpha:0.000100, R2:0.581, MSE:3.89, RMSE:1.97
Alpha:0.001000, R2:0.478, MSE:4.86, RMSE:2.20
Alpha:0.010000, R2:0.113, MSE:8.25, RMSE:2.87
Alpha:0.100000, R2:0.000, MSE:9.30, RMSE:3.05
Alpha:0.300000, R2:0.000, MSE:9.30, RMSE:3.05
Alpha:0.500000, R2:0.000, MSE:9.30, RMSE:3.05
Alpha:0.700000, R2:0.000, MSE:9.30, RMSE:3.05
Alpha:1.000000, R2:0.000, MSE:9.30, RMSE:3.05


In [26]:
elastic_net=ElasticNet(alpha=0.001, normalize=True).fit(X_tr,y_tr)
kfold_validate(X_tr, y_tr, test, elastic_net, 'elastic_net')

=========0-th Fold Score: 2.3059248184755434
=========1-th Fold Score: 2.194023310740103
=========2-th Fold Score: 1.9540169758009518
=========3-th Fold Score: 2.328617433222068
=========4-th Fold Score: 2.3125088042731172
=========Total Score:  2.2190182685023565


ValueError: shapes (4398,5047) and (50,) not aligned: 5047 (dim 1) != 50 (dim 0)

In [27]:
elastic_net_res = elastic_net.predict(X_val)
np.sqrt(mean_squared_error(y_val, elastic_net_res))

2.3974131915639347

## Lightgbm

In [ ]:
from lightgbm import LGBMRegressor

lgbm_reg = LGBMRegressor(n_estimator = 1000).fit(X,y)
kfold_validate(X_tr, y_tr, test, lgbm_reg, 'lightgbm_regression')

#LightGBMError: Do not support non-ascii characters in feature name.

In [ ]:
lgbm_res = lgbm_reg.predict(X_val)
np.sqrt(mean_squared_error(y_val, lgbm_res))

## XGBoost

In [18]:
import xgboost as xgb
xgb_reg = xgb.XGBRegressor()
kfold_validate(X_tr, y_tr, test, xgb_reg, 'xgboost')

=========0-th Fold Score: 1.7120325593942358


/Users/shuyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/shuyang/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/shuyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


=========1-th Fold Score: 1.7387320819003418
=========2-th Fold Score: 1.5388901893278666
=========3-th Fold Score: 1.8768504242084494
=========4-th Fold Score: 1.7765952602056343
=========Total Score:  1.7286201030073058


ValueError: feature_names mismatch: ['budget', 'name_collection', 'prodc_Québec Production Services Tax Credit', 'prodc_Abu Dhabi Film Commission', 'prodc_Colorado Office of Film, Television & Media', 'prodc_Chongoing Film Group', 'prodc_Bon Voyage Film Studio', 'prodc_Shanghai Media Group', 'prodc_Zhejiang Films & TV(Group) Company Ltd.', 'prodc_Hunan Broadcasting System', 'prodc_Anhui Broadcasting Corp.', 'prodc_Beijing TV Station', 'prodc_Centerstage Productions', 'prodc_Film Development Council of the Philippines', 'prodc_Percept Picture Company', 'budget_log', 'prodc_People Tree Films', 'prodc_Comet Film Produktion GmbH', 'prodc_Avrora Media', 'prodc_Cobblestone Pictures', 'prodc_An Erich von Stroheim Production', 'prodc_China Film Co.', 'prodc_Marvel Studios', 'prodc_Media Rights Capital (MRC)', 'prodc_One Race Films', 'prodc_Pitchblack Pictures Inc.', 'prodc_Lighthouse Pictures', 'prodc_Caliber Media Company', 'prodc_Motion Picture Corporation of America (MPCA)', 'prodc_Kim Ki-D

In [19]:
xgb_res = xgb_reg.predict(X_val)
np.sqrt(mean_squared_error(y_val, xgb_res))

2.0473965536487775